# Route optimization of a pub crawl with ORS and `ortools`

It's this of the year again (or will be in 6 months): the freshmen pour into the institute and as the diligent student council you are, you want to welcome them for their geo adventure with a stately pub crawl to prepare them for the challenges lying ahead.

We want to give you the opportunity to route the pack of rookies in a fairly optimal way:

In [1]:
conda create -n ors python=3.6 shapely
conda activate ors
pip install openrouteservice ortools
conda install -c conda-forge folium
conda install nb_conda_kernels

SyntaxError: invalid syntax (<ipython-input-1-0141c2c0c753>, line 1)

In [96]:
import folium
from shapely import wkt, geometry
import json
from pprint import pprint

## Visualize the study area

Now we're ready to start our most optimally planned pub crawl ever through hipster Kreuzberg! It will also be the most un-hipster pub crawl ever, as we'll cover ground with a taxi. At least it's safer than biking half-delirious.

First the basic parameters: API key and the district polygon to limit our pub search. The Well Known Text was prepared in QGIS from Berlin authority's [WFS](http://fbinter.stadt-berlin.de/fb/wfs/geometry/senstadt/re_ortsteil/) (QGIS field calculator has a `geom_to_wkt` method). BTW, Berlin, hope you don't wonder why your feature services are so slow... Simplify is the magic word, simplify.

Get your polygon as wkt string from https://arthur-e.github.io/Wicket/sandbox-gmaps3.html

In [2]:
wkt_str = 'Polygon ((8.589017443593661 49.454146402109245,8.758618884023349 49.454146402109245,8.758618884023349 49.35182374980772,8.589017443593661 49.35182374980772,8.589017443593661 49.454146402109245))'
aoi_geom = wkt.loads(wkt_str) # load geometry from WKT string

aoi_coords = list(aoi_geom.exterior.coords) # get coords from exterior ring
aoi_coords = [(y,x) for x,y in aoi_coords] # swap (x,y) to (y,x). Really leaflet?!
aoi_centroid = aoi_geom.centroid # Heidelberg center for map center

## Create a list of your route stops
We will use the Nominatim geocoder to find the coordinates of POIs in OSM. Use your email address as "user_agent".

In [66]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Christina.ludwig@uni-heidelberg.de")

In [132]:
# Function the switches the x and y coordinates, so that it is displayed correctly in the map ... just ignore it.
def switchCoords(coords):
    return (coords[1], coords[0])

In [113]:
stops = [{'coordinates' : switchCoords(geolocator.geocode("Wochenmarkt, Heidelberg, Germany")[1]), 'name': 'Christmas Market'},
            {'coordinates' : switchCoords(geolocator.geocode("Universitäts-Apotheke, Heidelberg, Germany")[1]), 'name': 'Pharmacy'},
            {'coordinates' : switchCoords(geolocator.geocode("Bismarckplatz, Heidelberg, Germany")[1]), 'name': 'Bismarkplatz'},
            {'coordinates' : switchCoords(geolocator.geocode("Zeughaus, Heidelberg, Germany")[1]), 'name': 'Marstall'},
            {'coordinates' : switchCoords(geolocator.geocode("Heidelberger Zuckerladen, Heidelberg, Germany")[1]), 'name': 'Zuckerladen'}]

In [114]:
stops

[{'coordinates': (8.71047601790124, 49.41212685), 'name': 'Christmas Market'},
 {'coordinates': (8.7045353, 49.4114141), 'name': 'Pharmacy'},
 {'coordinates': (8.69299861340895, 49.41001065), 'name': 'Bismarkplatz'},
 {'coordinates': (8.70515231355789, 49.4129347), 'name': 'Marstall'},
 {'coordinates': (8.7031265, 49.4093992), 'name': 'Zuckerladen'}]

## Create a map that shows all stops

Center map on Wochenmarkt by taking the coordinates of the first element in the list "stops"

In [133]:
centerOfMap = geometry.Point(stops[0]['coordinates'])

Create a map object and display it


In [134]:
# Create a map object
mapHD = folium.Map(tiles='Stamen Toner',location=(centerOfMap.y, centerOfMap.x), zoom_start=14)
# Display updated map
mapHD

Add the stops as Markers with a Popup to the map.

In [135]:
stopMarkers = []

for stop in stops:
    lon, lat = stop['coordinates']
    name = stop['name']
    popup = "<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(name, lat, lon)
    icon = folium.map.Icon(color='lightgray',
                        icon_color='#b5231a',
                        icon='star', # fetches font-awesome.io symbols
                        prefix='fa')
    folium.map.Marker([lat, lon], icon=icon, popup=popup).add_to(mapHD)
    stopMarkers.append(name)
# Display updated map
mapHD

## Find the optimal route connecting all stops
To determine the optimal route between the stops, we will use different ORS tools.

In [143]:
# Import all classes and functions that we need from the Open Route Service Python package
from openrouteservice import client, distance_matrix, directions
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

In order to use the OpenRouteService API you have to create and account and a free API key [here](https://openrouteservice.org/dev/#/login) After you created your API key, paste it below into the variable "api_key" in the second box below.

In [144]:
# Create a client using your API key that will execute all your requests to the ORS
api_key = '5b3ce3597851110001cf62483ae5cf98ae4743649600ff877943b339' #Provide your personal API key
myClient = client.Client(key=api_key)

In order to find the optimal route between, we first calculate the distances between all route stops. Set the routing profile (e.g. foot-walking, driving-car) and metric (duration or distance). For a list of available profiles see [Open Route Service Matrix Tool](https://openrouteservice.org/documentation/#/reference/matrix/matrix-service-(get))




In [147]:
profile = 'foot-walking'
metric = ['duration']

In [170]:
# Extract the coordinates from the list of stops
stop_coords = [feat['coordinates'] for feat in stops]
# Request to the ORS 
request = {'locations': stop_coords,
           'profile': profile,
           'metrics': metric}
# Execute the request
stops_matrix = clnt.distance_matrix(**request)

In [171]:
# Visualize the resulting stops_matrix 
import pandas as pd
stop_names = [st['name'] for st in stops]
df = pd.DataFrame(stops_matrix['durations'], columns=stop_names, index=stop_names)
df

,Christmas Market,Pharmacy,Bismarkplatz,Marstall,Zuckerladen
Christmas Market,0.00,345.77,1014.06,406.19,570.61
Pharmacy,345.77,0.00,668.30,165.83,252.74
Bismarkplatz,1014.06,668.30,0.00,800.05,629.67
Marstall,406.19,165.83,800.05,0.00,411.07
Zuckerladen,570.61,252.74,629.67,411.07,0.00


The code below finds the optimal route. Don't look at it in too much detail. It's not necessary to understand all of it 


In [172]:
def getDistance(from_id, to_id):
    return int(stations_matrix['durations'][from_id][to_id])

tsp_size = len(stop_names)
num_routes = 1
start = 0 # arbitrary start location

optimal_coords = []

if tsp_size > 0:
    routing = pywrapcp.RoutingModel(tsp_size, num_routes, start)
    search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
    # Create the distance callback, which takes two arguments (the from and to node indices)
    # and returns the distance between these nodes.
    routing.SetArcCostEvaluatorOfAllVehicles(getDistance)
    # Solve, returns a solution if any.
    assignment = routing.SolveWithParameters(search_parameters)
    if assignment:
        # Total cost of the 'optimal' solution.
        print("Total duration: " + str(round(assignment.ObjectiveValue(), 3) / 60) + " minutes\n")
        index = routing.Start(start) # Index of the variable for the starting node.
        route = ''
        # while not routing.IsEnd(index):
        for node in range(routing.nodes()):
            optimal_coords.append(stations_coords[routing.IndexToNode(index)])
            route += str(addresses[routing.IndexToNode(index)]) + ' -> '
            index = assignment.Value(routing.NextVar(index))
        route += str(addresses[routing.IndexToNode(index)])
        optimal_coords.append(stations_coords[routing.IndexToNode(index)])
        print("Route:\n" + route)

Total duration: 40.53333333333333 minutes

Route:
Christmas Market -> Marstall -> Bismarkplatz -> Zuckerladen -> Pharmacy -> Christmas Market


Visualizing both, the optimal route, and the more or less random waypoint order of the intial GeoJSON, look like this:

In [87]:
import os.path

def style_function(color):
    return lambda feature: dict(color=color,
                              weight=3,
                              opacity=1)

m = folium.Map(tiles='Stamen Toner',location=(aoi_centroid.y, aoi_centroid.x), zoom_start=14)
# See what a 'random' tour would have been
stations_coords.append(stations_coords[0])
request = {'coordinates': stations_coords,
           'profile': 'foot-walking',
           'geometry': 'true',
           'format_out': 'geojson',
#            'instructions': 'false'          
          }
random_route = clnt.directions(**request)

folium.features.GeoJson(data=random_route,
                        name='Random Route',
                        style_function=style_function('#84e184'),
                       overlay=True).add_to(m)

# And now the optimal route
request['coordinates'] = optimal_coords
optimal_route = clnt.directions(**request)
optRoute = folium.features.GeoJson(data=optimal_route,
                        name='Optimal Route',
                        style_function=style_function('#6666ff'),
                       overlay=True).add_to(m)

# add the layer control
m.add_child(folium.LayerControl())
m

### Export the optimal route to a geojson file

In [23]:
import json
with open("/Users/chludwig/Data/temp/optimalRoute.geojson", "w") as fp: 
    json.dump(optimal_route, fp)

The purple route looks a bit less painful. Let's see what the actual numbers say:

In [16]:
optimal_duration = 0
random_duration = 0

optimal_duration = optimal_route['features'][0]['properties']['summary'][0]['duration'] / 60
random_duration = random_route['features'][0]['properties']['summary'][0]['duration'] / 60
    
print("Duration optimal route: {0:.3f} mins\nDuration random route: {1:.3f} mins".format(optimal_duration,
                                                                                         random_duration))

Duration optimal route: 22.128 mins
Duration random route: 29.093 mins


The actual routing duration for the optimal order of venues is even minimally smaller than the optimization result (40.983 mins). Optimizing that route saved us a good 120€ worth of taxi costs.